In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("fifa21_raw_data.csv",low_memory=False)
df.head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,ID,Height,Weight,foot,BOV,BP,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,158023,"5'7""",159lbs,Left,93,RW,0,"Jul 1, 2004",NaN,€67.5M,€560K,€138.4M,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,20801,"6'2""",183lbs,Right,92,ST,0,"Jul 10, 2018",NaN,€46M,€220K,€75.9M,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,200389,"6'2""",192lbs,Right,91,GK,2,"Jul 16, 2014",NaN,€75M,€125K,€159.4M,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,192985,"5'11""",154lbs,Right,91,CAM,0,"Aug 30, 2015",NaN,€87M,€370K,€161M,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5 ★,4★,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,190871,"5'9""",150lbs,Right,91,LW,0,"Aug 3, 2017",NaN,€90M,€270K,€166.5M,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59,\n273


In [4]:
df['Positions'] = df['Positions'].str.split().str[0]

In [ ]:
df.head()

In [5]:
def clean_value_wage_clause(value):
    # Remove the '€' symbol
    value = value.replace('€', '')
    # Check if the value ends with 'M' and handle it
    if value.endswith('M'):
        value = value.replace('M', '')
        value = float(value) * 1e6  # Convert to numeric value, assuming 'M' means million
    elif value.endswith('K'):
        value = value.replace('K', '')
        value = float(value) * 1e3  # Convert to numeric value, assuming 'K' means thousand
    else:
        value = float(value)  # Convert to float if there's no 'M' or 'K'
    return value

In [6]:
columns = ['Value','Wage','Release Clause']
# Apply the function to each column in the list
for column in columns:
    df[column] = df[column].apply(clean_value_wage_clause)
    df[column] = df[column].astype(int)

In [7]:
# Dictionary for replacement
replacement_dict = {
    'RM': 'CM',
    'LM': 'CM',
    'RWB': 'RB',
    'LWB': 'LB',
    'CAM':'CM',
    'CF':'ST'
}

# Replace values in the DataFrame
df['Positions'] = df['Positions'].replace(replacement_dict)

In [ ]:
df['Positions'].unique()

In [ ]:
df.head()

In [8]:
df.rename(columns={"↓OVA": "Overall Rating"},inplace=True)

In [9]:
LP_data = df[['Name','Positions','Value','Age','Overall Rating','Height','Weight','foot','PAC','SHO','PAS','DRI','DEF','PHY']]

In [10]:
def get_optimized_team(df, max_budget, position_constraints):
    # Initialize variables and problem
    players_ids = df.index.tolist()
    overalls = dict(zip(players_ids, df['Overall Rating']))
    values = dict(zip(players_ids, df['Value']))
    positions = {pos: {i: 1 if pos == position else 0 for i, position in zip(players_ids, df['Positions'].values)} 
                 for pos in position_constraints.keys()}
    
    players_vars = LpVariable.dicts("Player", players_ids, cat='Binary')
    prob = LpProblem("OptimizedTeam", LpMaximize)
    
    # Objective function
    prob += lpSum([overalls[i] * players_vars[i] for i in players_ids])
    
    # Budget constraint
    prob += lpSum([values[i] * players_vars[i] for i in players_ids]) <= max_budget
    
    # Position and rating constraints
    for pos, constraints in position_constraints.items():
        min_overall = constraints['min_overall']
        max_overall = constraints['max_overall']
        
        for i in players_ids:
            if positions[pos][i] == 1:  # Player is eligible for this position
                # Add constraint for player's overall rating
                prob += (players_vars[i] * overalls[i] >= min_overall * players_vars[i])
                prob += (players_vars[i] * overalls[i] <= max_overall * players_vars[i])
        
        # Constraint for the count of players in this position
        prob += lpSum([positions[pos][i] * players_vars[i] for i in players_ids]) == constraints['count']
    
    # Constraint to ensure exactly 11 players are chosen
    prob += lpSum([players_vars[i] for i in players_ids]) == 11
    
    # Solve the problem using CBC solver
    prob.solve(PULP_CBC_CMD(msg=1))  # msg=1 displays CBC solver messages
    
    # Get the results
    selected_players = [int(player_id) for player_id in players_ids if players_vars[player_id].varValue == 1]
    selected_players_data = df.loc[selected_players]

    return LpStatus[prob.status], selected_players_data


In [12]:
# Set the budget
max_budget = 700000000  # 6,000,000,000

# Define your position constraints
position_constraints = {
    'ST': {'count': 1, 'min_overall': 70, 'max_overall': 100},
    'RB': {'count': 1, 'min_overall': 70, 'max_overall': 100},
    'LB': {'count': 1, 'min_overall': 70, 'max_overall': 100},
    'CB': {'count': 2, 'min_overall': 70, 'max_overall': 100},
    'CM': {'count': 2, 'min_overall': 70, 'max_overall': 100},
    'CDM':{'count': 1, 'min_overall': 70, 'max_overall': 100},
    'LW': {'count': 1, 'min_overall': 70, 'max_overall': 100},
    'RW': {'count': 1, 'min_overall': 70, 'max_overall': 100},
    'GK': {'count': 1, 'min_overall': 70, 'max_overall': 100}
}

# Call the function with the DataFrame, budget, and constraints
status_code, selected_players_data = get_optimized_team(LP_data, max_budget, position_constraints)

# Check if status_code is an integer (typical case) or a string (unusual case)
if isinstance(status_code, int):
    # Translate the status code to a human-readable message (typical case)
    status_message = LpStatus[status_code]
else:
    # Directly use the status_code as the message (unusual case)
    status_message = status_code

print("Status:", status_message)
if status_message == 'Optimal':
    print("Selected players:")
    print(selected_players_data)
else:
    print("No optimal solution was found.")

Status: Optimal
Selected players:
                   Name Positions     Value  Age  Overall Rating Height  \
0              L. Messi        RW  67500000   33              93   5'7"   
1     Cristiano Ronaldo        ST  46000000   35              92   6'2"   
2              J. Oblak        GK  75000000   27              91   6'2"   
3          K. De Bruyne        CM  87000000   29              91  5'11"   
4             Neymar Jr        LW  90000000   28              91   5'9"   
10          V. van Dijk        CB  75500000   28              90   6'4"   
12             Casemiro       CDM  59500000   28              89   6'1"   
16         Sergio Ramos        CB  24500000   34              89   6'0"   
26             T. Kroos        CM  55000000   30              88   6'0"   
29  T. Alexander-Arnold        RB  60000000   21              87  5'11"   
31         A. Robertson        LB  53000000   26              87  5'10"   

    Weight   foot  PAC  SHO  PAS  DRI  DEF  PHY  
0   159lbs   Le

In [13]:
# Set the budget
max_budget = 70000000  # 6,000,000,000

# Define your position constraints
position_constraints = {
    'ST': {'count': 1, 'min_overall': 60, 'max_overall': 100},
    'RB': {'count': 1, 'min_overall': 60, 'max_overall': 100},
    'LB': {'count': 1, 'min_overall': 60, 'max_overall': 100},
    'CB': {'count': 2, 'min_overall': 60, 'max_overall': 100},
    'CM': {'count': 2, 'min_overall': 60, 'max_overall': 100},
    'CDM':{'count': 1, 'min_overall': 60, 'max_overall': 100},
    'LW': {'count': 1, 'min_overall': 60, 'max_overall': 100},
    'RW': {'count': 1, 'min_overall': 60, 'max_overall': 100},
    'GK': {'count': 1, 'min_overall': 60, 'max_overall': 100}
}

# Call the function with the DataFrame, budget, and constraints
status_code, selected_players_data = get_optimized_team(LP_data, max_budget, position_constraints)

# Check if status_code is an integer (typical case) or a string (unusual case)
if isinstance(status_code, int):
    # Translate the status code to a human-readable message (typical case)
    status_message = LpStatus[status_code]
else:
    # Directly use the status_code as the message (unusual case)
    status_message = status_code

print("Status:", status_message)
if status_message == 'Optimal':
    print("Selected players:")
    print(selected_players_data)
else:
    print("No optimal solution was found.")

Status: Optimal
Selected players:
               Name Positions     Value  Age  Overall Rating Height  Weight  \
27    S. Handanovič        GK  16000000   35              88   6'4"  203lbs   
47     G. Chiellini        CB  15500000   35              87   6'2"  187lbs   
140     Jesús Navas        RB  11000000   34              84   5'8"  132lbs   
213  Z. Ibrahimović        ST   9500000   38              83   6'5"  209lbs   
288  Welington Dano        LB         0   20              81  5'10"  152lbs   
292  Juiano Mestres        CB         0   24              81  5'11"  181lbs   
370      J. Sildero        CM         0   28              80  5'10"  165lbs   
374       S. Ardero        CM         0   32              80   5'7"  139lbs   
375      M. Baldona       CDM         0   36              80  5'10"  165lbs   
475            Hulk        RW   9000000   33              80  5'11"  187lbs   
502            Nani        LW   9000000   33              80  5'10"  154lbs   

      foot  PAC  